In [13]:
using StatsBase
using Combinatorics
include("jl/vol.jl");

In [14]:
# toy data

Z = [1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5] # group partition
D = [3, 4, 2, 5, 6, 4, 3, 2, 5, 2, 2]; # degree sequence

First: let's check the formula

$$\sum_{R \in [n]^\ell} \mathbb{I}(\#(\mathbf{z}_R) = p) \sigma(\theta_R) = \sum_{y: \#y = p}\prod_{a = 1}^{\ell}v_{y_a}\;,$$

In this formula, we're allowing R to range over $[n]^\ell$, where $\ell$ is the number of nodes per hyperedge, and $n$ the number of nodes.  $p$ is a specified permutation. This is **different** from the current version of the nodes, and matches the conversation we had over email. 

The function `evalSumNaive` performs the summation on the lefthand side, while the function `evalSumPV` uses the product-of-volumes representation on the righthand side. Both of these functions are defined in `jl/sums.jl`.



In [15]:
# test: check that these two functions give the same result on all partitions
# this will be very slow for even moderate n and r

r = 3

for i = 1:r, j = 1:i, p in partitions(i,j)
    println(evalSumNaive(p, Z, D) == evalSumPV(p, Z, D))
end

true
true
true
true
true
true


Ok, looks good! `evalSumPV` is a lot faster than `evalSumNaive`, although they are both very slow. Presumably this is due in part to kludgy coding on my part, but one would imagine that even much better coding practice could only improve these so much

In [17]:
p = [2, 1]
@time evalSumNaive(p, Z, D)
@time evalSumPV(p, Z, D)

  0.094113 seconds (30.42 k allocations: 94.904 MiB, 27.77% gc time)
  0.004917 seconds (2.65 k allocations: 8.912 MiB)


27546

We can use `evalSumsPV` to compute the required sum for each partition. In principle, one could do this with `evalSumsNaive` as well (also implemented), but the latter is VERY slow. 

In [19]:
# need to run twice to avoid timing compile times
r = 5 # size of largest hyperedge
@time MPV = evalSumsPV(Z, D, r)

  1.294968 seconds (504.27 k allocations: 1.791 GiB, 26.14% gc time)


Dict{Any,Any} with 18 entries:
  [2, 2, 1]       => 23050800
  [1, 1, 1, 1]    => 346080
  [3, 2]          => 6288620
  [2, 2]          => 220614
  [3]             => 2750
  [1, 1]          => 1130
  [1, 1, 1]       => 24576
  [2]             => 314
  [2, 1, 1]       => 1175616
  [2, 1]          => 27546
  [4, 1]          => 3587830
  [4]             => 25058
  [1]             => 38
  [5]             => 234638
  [2, 1, 1, 1]    => 26997600
  [3, 1, 1]       => 16723680
  [1, 1, 1, 1, 1] => 2352000
  [3, 1]          => 317768

Remembering that n = 10 in this case and that we will need to evaluate these sums many many times, this timing is not practical. 

Now we'll use the `evalSums`, which uses a recursion lemma from the notes in order to compute the relevant sums for every partition vector p at once. Rather than a simple loop over all partitions like `evalSumsPV` uses, `evalSums` actually uses previously calculated values in order to significantly reduce the per-partition compute time. 

In [22]:
# need to run this block twice in order to avoid timing compile time
@time V, μ, M = evalSums(Z, D, r);

  0.000745 seconds (2.00 k allocations: 799.281 KiB)


The result agrees with `evalSumsPV` on all partitions, but is roughly 10,000 times as fast. 

# How big can we go?

Even with my highly non-optimized code, we can do medium-sized instances with large hyperedges fairly quickly this way. Note, however, that we need BigInts to avoid overflow issues. 

With this method, we can compute on n = 50,000 nodes and hyperedges of up to size 20 in roughly the same compute time that it took `evalSumsPV` to do 10 nodes and hyperedges up to size 5, before further optimization. In a later test (not shown), I was able to compute on 1M nodes in under a minute. 

In [25]:
# Performance test: how big can we do this?
n = 50000

Z = rand(1:50, n)
D = rand(2:100, n)

r = 10 # maximum hyperedge size

@time V, μ, M = evalSums(Z, D, r;constants=true, bigInt=true);

  1.405072 seconds (10.27 M allocations: 222.936 MiB, 22.34% gc time)


# Calculating Local Updates

We'd now like to see if we can use this method to compute the change in the entries of M associated with moving node $i$ from group $s$ to group $t$. In principle, all we need to do is a bit of bookkeeping that will allow us to propagate this change through the entries of $M$ in a fast way. This could potentially be a foundation for a Louvain-style approach, in which we would then need to evaluate the total impact of a move and choose from among a set of possibilities to find the best one. 

Let's first go back to our toy data: 

In [36]:
# toy data

Z = [1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5] # group partition
D = [3, 4, 2, 5, 6, 4, 3, 2, 5, 2, 2]; # degree sequence

r = 5

5

Here's a function that we'll use to test our updates. It first calculates N for the specified partition set, and swaps nodes between groups randomly and evaluates the change in $N$ associated with each swap, for a specified number of rounds. Choosing `check=true` has it compare the result to the ground-truth sums N computed on the modified grouping vector. 

In [37]:
function testUpdates(Z, D, r, rounds=100, check=true, bigInt=false)
    
    ℓ = maximum(Z) # number of total groups
    
    V, μ, M = evalSums(Z, D, r;constants=false, bigInt=bigInt);
    C = evalConstants(r)

    N = Dict()

    for k in 1:rounds

        # random proposal swap
        i = rand(1:length(D)) # node to move
        t = rand(1:ℓ) # new group
        s = Z[i] # old group

        # increments due to proposal
        ΔV, Δμ, ΔM = increments(V, μ, M, i, t, D, Z);

        # new quantities (assumes we accept every proposal)
        V, μ, M = addIncrements(V, μ, M, ΔV, Δμ, ΔM)

        # carry out the change in membership
        Z[i] = t

        # multiply by combinatorial factors to get the sums we actually want
        N = Dict(p => M[p]*C[p] for p in keys(M))
    end
    if check
        V̄, μ̄, N̄ = evalSums(Z, D, r; constants=true)
        Dict(p => N[p] == N[p] for p in keys(M))
    end     
end;

Let's choose `check=true`, so we can compare to $N$ as computed from scratch. We are looking for these to agree on all partitions. 

In [38]:
testUpdates(Z, D, 5, 100, true, true)

Dict{Array{Integer,1},Bool} with 18 entries:
  Integer[2, 2, 1]       => true
  Integer[1, 1, 1, 1]    => true
  Integer[3, 2]          => true
  Integer[2, 2]          => true
  Integer[3]             => true
  Integer[1, 1]          => true
  Integer[1, 1, 1]       => true
  Integer[2]             => true
  Integer[2, 1, 1]       => true
  Integer[2, 1]          => true
  Integer[4, 1]          => true
  Integer[4]             => true
  Integer[1]             => true
  Integer[5]             => true
  Integer[2, 1, 1, 1]    => true
  Integer[3, 1, 1]       => true
  Integer[1, 1, 1, 1, 1] => true
  Integer[3, 1]          => true

Looks ok! Now let's try to get a sense for how we can do on a larger instance. 

In [43]:
n = 50000
Z = rand(1:50, n)
D = rand(2:100, n)
r = 10 # maximum hyperedge size

10

In [44]:
n_steps = 1000
@time testUpdates(Z, D, r, n_steps, false, true);

  4.145152 seconds (30.72 M allocations: 903.154 MiB, 16.28% gc time)


Recall that, on an instance of this size, it took us ~1.5s to compute all the required sums from scratch. On the other hand, we can maintain bookkeeping through 1000 updates for only 3s more, which is a pretty good computational savings. I'm pretty convinced that these results can be improved considerably by better coding. It's also worth noting that the size of the hyperedge is responsible for much of the scaling behavior: reducing `r` significantly reduces the runtime. 